# Dipole Field Analysis in 3D
This notebook analyzes synthetic magnetic dipole fields with added noise and sampling effects.
It demonstrates:
- Dipole generation
- Gaussian and 1/f noise
- Aliasing via undersampling
- Low-pass filtering
- FFT visualization of each condition

In [ ]:
# Imports and Grid Setup
import numpy as np
from numpy.fft import fftn, fftshift, ifftn
from scipy.ndimage import gaussian_filter

N = 32
step = 4
x = np.linspace(-1, 1, N)
y = np.linspace(-1, 1, N)
z = np.linspace(-1, 1, N)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

In [ ]:
# Dipole Field Function
def magnetic_dipole_field(x, y, z, x0, y0, z0, m):
    r = np.sqrt((x - x0)**2 + (y - y0)**2 + (z - z0)**2) + 1e-6
    r_hat_x = (x - x0) / r
    r_hat_y = (y - y0) / r
    r_hat_z = (z - z0) / r
    dot = m * r_hat_z
    Bx = (3 * r_hat_x * dot) / (r**3)
    By = (3 * r_hat_y * dot) / (r**3)
    Bz = (3 * r_hat_z * dot - m) / (r**3)
    return Bx, By, Bz

In [ ]:
# 1/f Noise Generator
def generate_1f_noise(shape, amplitude_scale=1.0):
    kx = np.fft.fftfreq(shape[0]).reshape(-1, 1, 1)
    ky = np.fft.fftfreq(shape[1]).reshape(1, -1, 1)
    kz = np.fft.fftfreq(shape[2]).reshape(1, 1, -1)
    k_mag = np.sqrt(kx**2 + ky**2 + kz**2)
    k_mag[0, 0, 0] = 1e-6
    spectrum = np.random.normal(size=shape) + 1j * np.random.normal(size=shape)
    spectrum *= (1.0 / k_mag)
    noise = np.real(ifftn(spectrum))
    noise -= np.mean(noise)
    noise /= np.max(np.abs(noise))
    return noise * amplitude_scale

In [ ]:
# Field Generation
dipoles = [
    {"pos": (-2.5, 0.7, -1.2), "m": 1.0},
    {"pos": (-0.5, -0.5, -0.5), "m": 0.3},
    {"pos": (0.5, 0.5, 0.5), "m": 0.5},
    {"pos": (-0.7, 0.6, 0.0), "m": 0.2}
]
baseline_vector = (-0.3, -0.5, -0.7)

Bx = By = Bz = np.zeros_like(X)
for d in dipoles:
    bx, by, bz = magnetic_dipole_field(X, Y, Z, *d["pos"], d["m"])
    Bx += bx
    By += by
    Bz += bz
Bx += baseline_vector[0]
By += baseline_vector[1]
Bz += baseline_vector[2]

In [ ]:
# Add Gaussian and 1/f Noise
smallest_m = min(d["m"] for d in dipoles)
np.random.seed(42)
gauss_1p = 0.01 * smallest_m
gauss_10p = 0.10 * smallest_m
pink_5p = 0.05 * smallest_m

Bx_10p = Bx + np.random.normal(0, gauss_10p, Bx.shape)
By_10p = By + np.random.normal(0, gauss_10p, By.shape)
Bz_10p = Bz + np.random.normal(0, gauss_10p, Bz.shape)

pink_noise = generate_1f_noise(Bx.shape, pink_5p)
Bx_combined = Bx_10p + pink_noise
By_combined = By_10p + pink_noise
Bz_combined = Bz_10p + pink_noise

In [ ]:
# FFT Utility
def fft_mag(Bx, By, Bz):
    return fftshift(np.abs(fftn(np.sqrt(Bx**2 + By**2 + Bz**2))))

# Example usage
fft_combined = fft_mag(Bx_combined, By_combined, Bz_combined)